In [2]:
import cv2, imp, os
from PIL import Image, ImageDraw
import numpy as np

In [12]:
print(cv2.__version__)

jpg_root = './../../lc_files/20181218_jpg/469LC1183340002/page_1.jpg'


img = cv2.imread(jpg_root, 0)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img = cv2.bitwise_not(img)
# th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv2.THRESH_BINARY,11,2)

ret, th = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

image = Image.fromarray(th)
image.show()

# thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
# coords = np.column_stack(np.where(thresh > 0))
# angle = cv2.minAreaRect(coords)[-1]
# if angle < -45:
#     angle = -(90 + angle)
# else:
#     angle = -angle
# (h, w) = img.shape[:2]
# center = (w // 2, h // 2)
# M = cv2.getRotationMatrix2D(center, angle, 1.0)
# rotated = cv2.warpAffine(img, M, (w, h), 
#                          flags=cv2.INTER_CUBIC, 
#                          borderMode=cv2.BORDER_REPLICATE) 
# print("[INFO] angle: {:.3f}".format(angle))
# rotated = cv2.bitwise_not(rotated)
# image = Image.fromarray(rotated)
# image.show()

# image = Image.fromarray(erosion)
# # image.show()
# saved_path = "./cvtest.jpg"
# image.save(saved_path, "JPEG")

3.4.4


In [5]:
import package.utils as utils
import package.visionapi as visionapi
import package.augmentation as imgaug
import service

imp.reload(utils)
imp.reload(visionapi)
imp.reload(imgaug)
imp.reload(service)

jpg_root = './../../lc_files/20181218_jpg/MF0IML183330002'
jpg_list = ['./../../lc_files/20181218_jpg/MF0IML183330002/page_1.jpg', 
            './../../lc_files/20181218_jpg/MF0IML183330002/page_2.jpg', 
            './../../lc_files/20181218_jpg/MF0IML183330002/page_3.jpg', 
            './../../lc_files/20181218_jpg/MF0IML183330002/page_4.jpg', 
            './../../lc_files/20181218_jpg/MF0IML183330002/page_5.jpg']

credential = '../../service_acc.json'
visionapi.set_vision_credential(credential)
# res = visionapi.annotateDocument(saved_path)
service.annotateCreditLetter(credential, '0001', jpg_list, jpg_root)

{'checklist': {'additional_docs': {'original_47A': '1. TRANSPORT DOCUMENTS BEARING REFERENCE BY STAMP OR OTHERWISE TO\nCOST ADDITIONAL TO THE FREIGHT CHARGES ARE NOT ACCEPTABLE.\n2. ALL DOCUMENTS SHOULD QUOTE OUR CREDIT NUMBER AND DATE.\n3. DECK SHIPMENT PROHIBITED.\n4. ALL DOCUMENTS SHOULD BE IN ENGLISH.\n5. IF BILLS ARE RECEIVED WITH DISCREPANCIES, A DISCREPANCY FEE OF\nUSD 75.00 WILL BE CHARGED IF THE DISCREPANCIES ARE ACCEPTED BY US\nAND THE APPLICANT WHICH WILL BE DEDUCTED FROM THE BILL PROCEEDS.\n6. DUE DATE SHOULD NOT EXCEED 360 DAYS FROM THE DATE OF SHIPMENT.\n7. SEPERATE SET OF COMMERCIAL INVOICE SHOULD BE SUBMITTED FOR\nEACH BILL OF LADING IF PART SHIPMENT IS EFFECTED.\n8. ALL DOCUMENTS DATED PRIOR TO LC ISSUE DATE ARE NOT ACCEPTABLE,\n8.USANCE INTEREST IS PAYABLE BY THE APPLICANT AT 6M LIBOR PLUS\n110 BASIS POINTS PER ANNUM + USD 200 AS DOCUMENT HANDLING CHARGESAND SWIFT CHARGES FOR PER SET OF DOCUMENTS.\n10.THE PRESENTING BANK HAS TO DESPATCH THE DOCUMENTS WITH\nORIGINAL LC

In [65]:
tmp = os.path.basename(jpg_root)
print(tmp)

page_4.jpg
